# Finding the best heuristic weights with a GA

In [125]:
import subprocess
import random
import numpy as np
from subprocess import check_output

In [91]:
%pip install geneticalgorithm

     |████████████████████████████████| 44 kB 3.0 MB/s eta 0:00:011
  Created wheel for func-timeout: filename=func_timeout-4.3.5-py3-none-any.whl size=15077 sha256=bce8704a6374444a032759c63786282c32c472c3d8a27146e8a31ea3f7280074
  Stored in directory: /home/macdue/.cache/pip/wheels/a8/92/ca/5bbab358275e310af23b73fc32ebf37d6a7a08c87c8d2cdbc1
Successfully built func-timeout
Note: you may need to restart the kernel to use updated packages.


In [92]:
from geneticalgorithm import geneticalgorithm as ga

In [13]:
OUR_BOT = "cargo run --release -- --search=alpha-beta --weights {} --depth {}"
OPP_BOT = "java -jar ../MKRefAgent.jar"

In [119]:
def execute(cmd):
    cmd = ' '.join(cmd)
    print(cmd)
    popen = subprocess.Popen(cmd, stderr=subprocess.PIPE, universal_newlines=True, shell=True)
    for line in iter(popen.stderr.readline, ""):
        yield line 
    popen.stderr.close()
    return_code = popen.wait()
    if return_code:
        raise subprocess.CalledProcessError(return_code, cmd)

In [126]:
def run_bot(start_south, weights, depth = 10):
    if len(weights) != 6:
        raise Exception("6 weights needed")
    p1 = OUR_BOT.format(' '.join(map(str, weights)), 10)
    p2 = OPP_BOT
    if not start_south:
        p2, p1 = p1, p2
    try:
        return [*execute(["java", "-jar", "../ManKalah.jar", f'"{p1}"', f'"{p2}"'])]
    except subprocess.CalledProcessError as run_e:
        print(run_e.returncode, run_e.output)
        raise run_e

In [121]:
def parse_result(res):
    p1_info  = res[-3].rsplit(':')[1].split()
    p1_moves = int(p1_info[0])
    p1_time = int(p1_info[2])
    
    p2_info  = res[-2].rsplit(':')[1].split()
    p2_moves = int(p2_info[0])
    p2_time = int(p2_info[2])
    
    score_line = res[-6]
    
    if "DRAW" in score_line:
        winner = 0
    else:
        winner = int(score_line.split()[2]);
        if winner == 2:
            winner = -1
    
    return {
        "winner":winner,
        "score": int(res[-5].split()[1]),
        "p1_turns": p1_moves,
        "p1_time": p1_time,
        "p2_turns": p2_moves,
        "p2_time": p2_time
    }

In [133]:
def f(X):
    start_south = bool(random.getrandbits(1))
    res = parse_result(run_bot(start_south, X[:6], 10))
    winner = res["winner"] * 1 if start_south else -1
    res["ga_score"] = res["score"] * -winner
    print(res)
    return res["ga_score"] # ga finds min

In [134]:
vartype = np.array([['real'],['real'],['real'],['real'],['real'],['real']])
varbound = np.array([[0.0,1.0],[0.0,1.0],[0.0,1.0],[0.0,1.0],[0.0,1.0],[0.0,1.0]])
model = ga(function=f, dimension=6, variable_type_mixed=vartype, variable_boundaries=varbound)

In [135]:
model.run()

java -jar ../ManKalah.jar "java -jar ../MKRefAgent.jar" "cargo run --release -- --search=alpha-beta --weights 0.14548474219467833 0.07754806077579779 0.8238760552310512 0.5498673281020952 0.5907346904822612 0.5946231395436626 --depth 10"
{'winner': -1, 'score': 22, 'p1_turns': 32, 'p1_time': 0, 'p2_turns': 34, 'p2_time': 0, 'ga_score': 22}
java -jar ../ManKalah.jar "java -jar ../MKRefAgent.jar" "cargo run --release -- --search=alpha-beta --weights 0.8165715146109185 0.15154461091636373 0.277212157086164 0.17489613175742091 0.27437266804609917 0.7916810470602729 --depth 10"
{'winner': 1, 'score': 28, 'p1_turns': 18, 'p1_time': 0, 'p2_turns': 20, 'p2_time': 0, 'ga_score': 28}
java -jar ../ManKalah.jar "java -jar ../MKRefAgent.jar" "cargo run --release -- --search=alpha-beta --weights 0.984811800472968 0.4739181276512976 0.040696612924242204 0.2322724364452594 0.44614112745717505 0.39286974526349405 --depth 10"
{'winner': -1, 'score': 8, 'p1_turns': 25, 'p1_time': 0, 'p2_turns': 29, 'p2_t

KeyboardInterrupt: 

 None
